# Analyze Results

In [ ]:
import os.path
import numpy as np
import pandas as pd
from matplotlib import pyplot

from hit_prediction_code.analytics import get_results_as_dataframe
from hit_prediction_code import analytics
from sklearn.metrics import ConfusionMatrixDisplay

results = get_results_as_dataframe(
    project_name='hit-prediction-ismir2020',
    table_name='hit_prediction',
    filter_git_dirty=True,
#     date_filter='BETWEEN \'2021-04-07 14:00:00\' AND \'2021-04-13 17:00:00\'',
#     date_filter='>= \'2022-03-24 14:21:00\'',
    date_filter='>= \'2022-04-07 10:35:00\'',
    columns=['id', 'sourcefile', 'outcome'],
#     filters=['sourcefile LIKE \'plans/wide_and_deep_performance/%%ordinal_one_hot_all_lfmlc.py\''],
    filters=['sourcefile LIKE \'plans/hsppw/qcut_hsp-s%%.py\''],
#     filters=['sourcefile LIKE \'plans/ordinal%%\''],
)

analytics.add_approach_to_df(results)
analytics.add_cv_epoch_evaluator_outcome_to_df(results)

results.sort_values(by='sourcefile', inplace=True)
display(len(results))

In [ ]:
def compute_mae_from_cm(matrix, average='macro'):
    assert matrix.shape[0] == matrix.shape[1]
       
    if average == 'macro':
        distance = matrix * _distance_multiplyer(matrix.shape[0])
        
        class_distance = np.sum(distance, axis=1)
        class_counts = np.sum(matrix, axis=1)
        
        mask = class_counts > 0
        
        return np.mean(class_distance[mask] / class_counts[mask])

def _distance_multiplyer(n):
    m = []
    for i in range(n):
        r = range(-i, n-i, 1)
        m.append(r)
    
    return abs(np.array(m))

def compute_kappa_from_cm(matrix):
    cidot = matrix.sum(axis=1)
    cdotj = matrix.sum(axis=0)
    N = matrix.sum()
    
    def w(i, j):
        return abs(i - j)
    
    def e(i, j):
        return cidot[i] * cdotj[j] / N
        
    def c(i, j):
        return matrix[i, j]
    
    nom = 0
    dis = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            nom += (w(i,j) * c(i, j))
            dis += (w(i,j) * e(i, j))
    
    return 1 - nom / dis
    

In [ ]:
def split_names(num_splits):
    return ['split-' + str(i) for i in range(5)]


def compute_score_on_cv(result, num_splits):
    mean_col = 'mean'
    splits = split_names(num_splits)
    cm_name = 'confusion_matrix'
    
    assert cm_name in result[mean_col].index
    assert len(result[mean_col].loc[cm_name]) == 1
    
    def compute_scores(col):
        cm = np.array(result[col].loc[cm_name][0])
        mcm = analytics.confusion_matrix_to_multilabel_confusion_matrix(cm)
        scores = analytics.scores_from_multilabel_confusion_matrix(mcm)
        
        scores['macro_mae'] = compute_mae_from_cm(cm)
        scores['macro_kappa'] = compute_kappa_from_cm(cm)
        
        return scores
        
    mean = compute_scores(mean_col)
    
    split_scores = {}
    for metric in mean.keys():
        split_scores[metric] = []
    
    for split in splits:
        scores = compute_scores(split)
        for metric, score in scores.items():
            split_scores[metric].append(score)
    
    split_mean = {}
    for metric, scores in split_scores.items():
        split_mean[metric] = np.mean(scores)
    
    return mean, split_mean

result_scores = []

for _, result in results.iterrows():
    approach = result['approach'].split(' ')
    infos = approach[1].split('_')
       
    result_entry = {
            'approach_id': approach[0],
            'pair': infos[0],
            'data': infos[1],
            'classes': infos[2],
            'approach': infos[3],
            'model': infos[4],
    }
    
    id_cols = [c for c in result_entry.keys() if c != 'approach_id']
    
    mean, split_mean = compute_score_on_cv(result, 5)
    
    for metric, score in mean.items():
        result_entry['mean_' + metric] = score
    
    
    for metric, score in split_mean.items():
        result_entry['split_mean_' + metric] = score     
    
    result_scores.append(result_entry)

    
df = pd.DataFrame(result_scores)
df.sort_values(by=['model', 'approach', 'classes']).drop_duplicates(subset=id_cols).to_csv('/tmp/test.csv')

In [ ]:
# normalize = 'true'
normalize = False

for _, result in results.iterrows():
    mean_result = result['mean']
    
    if 'confusion_matrix' in mean_result.index:
        normalized_cm = []
        for cm in mean_result.loc['confusion_matrix']:
            if normalize:
                cm = analytics.normalize_confusion_matrix(cm, method=normalize)
            else:
                cm = np.array(cm)

        normalized_cm.append(cm)
        display(result['approach'])
        outcome = analytics.scores_from_confusion_matrices(normalized_cm, mean_result.columns)
        display(outcome)
        best_f1 = outcome[outcome['macro_f1'] >= outcome['macro_f1'].max()]
#         display(best_f1)

        plot_shape = (1, 1)
        fig_width = 12
        if len(normalized_cm) >= plot_shape[0] * plot_shape[1]:
            fig_size = (fig_width, int(fig_width / plot_shape[1] * plot_shape[0] * 0.85))
#             analytics.plot_epochs_confution_matrix(result['approach'], normalized_cm, mean_result.columns, plot_shape=plot_shape, figsize=fig_size)

#         analytics.plot_reg(np.array(mean_result.loc['confusion_matrix'][best_f1.index[0]]))
        
    elif 'multilabel_confusion_matrix' in mean_result.index:
        for mcm in mean_result.loc['multilabel_confusion_matrix']:
#             for cm in mcm:
#                 disp = ConfusionMatrixDisplay(np.array(cm))
#                 disp.plot()
#                 pyplot.show()
            mcm = np.array(mcm)
            display(mcm)
            display(analytics.scores_from_multilabel_confusion_matrix(mcm))
